In [1]:
# 2-3d scatterplot (Credit to Microsoft Copilot)
# Note to self: figure out how the heck this works)
# To do: something ratios
import dash
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output, State, ctx, ALL
import plotly.graph_objects as go
import plotly.express as px
import os
import signal
%run -i assets/lists.ipynb

# imports Planetary systems csv
pl_s = pd.read_csv('assets/Planetary_Systems.csv', comment="#", low_memory=False)
pl_es = pd.read_csv('assets/Planetary_Systems_Estimated.csv', comment="#", low_memory=False)

# imports atmospheric list for one filter.
at_s = pd.read_csv('assets/Atmospheric_Spectroscopy.csv', comment="#", low_memory=False)

solar_planets = pd.read_csv('assets/Solar_Values.csv', comment="#", low_memory=False)

# Dash app
app = Dash(__name__, external_stylesheets=['assets/style.css'])

# List of database columns to take into account for graphs
planet_features = {
    'Planet (Compare Planets only)': [
        {'label': "Planet Orbital Period (days, pl_orbper, Recommended)", 'value': "pl_orbper"},
        {'label': "Planet Orbit Semi-Major Axis (au, pl_orbsmax)", 'value': "pl_orbsmax"},
        {'label': "Planet Epoch of Periastron (deg, pl_orbtper)", 'value': "pl_orbtper"},
        {'label': "Planet Argument of Periastron (deg, pl_orblper)", 'value': "pl_orblper"},
        {'label': "Planet Projected Obliquity (deg, pl_projobliq)", 'value': "pl_projobliq"},
        {'label': "Planet True Obliquity (deg, pl_trueobliq)", 'value': "pl_trueobliq"},
        {'label': "Planet Radius (Earth radius, pl_rade, Recommended)", 'value': "pl_rade"},
        {'label': "Planet Mass (Earth mass, pl_bmasse, Recommended)", 'value': "pl_bmasse"},
        {'label': "Planet Density (g/cm^3, pl_dens, Recommended)", 'value': "pl_dens"},
        {'label': "Planet Orbital Eccentricity (pl_orbeccen)", 'value': "pl_orbeccen"},
        {'label': "Planet Insolation Flux (Earth flux, pl_insol)", 'value': "pl_insol"},
        {'label': "Planet Equilibrium Temperature (K, pl_eqt, Recommended)", 'value': "pl_eqt"},
        {'label': "Planet Transit Duration (hrs, pl_trandur)", 'value': "pl_trandur"},
        {'label': "Planet Transit Midpoint (days, pl_tranmid)", 'value': "pl_tranmid"},
        {'label': "Planet Transit Depth (%, pl_trandep)", 'value': "pl_trandep"},
        {'label': "Planet Impact Parameter (pl_imppar)", 'value': "pl_imppar"},
        {'label': "Planet Occulation Depth (%, pl_occdep)", 'value': "pl_occdep"},
        {'label': "Planet Radial Velocity Amplitude (m/s, pl_rvamp)", 'value': "pl_rvamp"},
        {'label': "Discovery Year (disc_year)", 'value': "disc_year"},
        {'label': "Last Update (rowupdate)", 'value': "rowupdate"},
    ],
    'Stellar': [
        {'label': "Stellar Effective Temperature (K, st_teff, Recommended)", 'value': "st_teff"},
        {'label': "Stellar Radius (Solar radius, st_rad, Recommended)", 'value': "st_rad"},
        {'label': "Stellar Mass (Solar mass, st_mass, Recommended)", 'value': "st_mass"},
        {'label': "Stellar Density (g/cm^3, st_dens)", 'value': "st_dens"},
        {'label': "Stellar Surface Gravity (log10(cm/s^2), st_logg)", 'value': "st_logg"},
        {'label': "Stellar Age (Gyr, st_age)", 'value': "st_age"},
        {'label': "Stellar Rotational Period (days, st_rotp)", 'value': "st_rotp"},
        {'label': "Stellar Rotational Velocity (km/s, st_vsin)", 'value': "st_vsin"},
        {'label': "Stellar Radial Velocity (km/s, st_radv)", 'value': "st_radv"},
        {'label': "Stellar Metallicity (dex, st_met, Recommended)", 'value': "st_met"},
        {'label': "Stellar Luminosity (log10(Solar), st_lum)", 'value': "st_lum"},
    ],
    'System': [
        {'label': "System Parallax (mas, sy_plx)", 'value': "sy_plx"},
        {'label': "System Distance from Earth (pc, sy_dist)", 'value': "sy_dist"},
        {'label': "System # Stars (sy_snum)", 'value': "sy_snum"},
        {'label': "System # Planets (sy_snum)", 'value': "sy_pnum"},
        {'label': "System #  Moons (sy_snum)", 'value': "sy_mnum"},
        {'label': "System u (Sloan) Magnitude (sy_umag, ~354 nm)", 'value': "sy_umag"},
        {'label': "System B (Johnson) Magnitude (sy_bmag, ~442 nm)", 'value': "sy_bmag"},
        {'label': "System g (Sloan) Magnitude (sy_gmag, ~475 nm)", 'value': "sy_gmag"},
        {'label': "System V (Johnson) Magnitude (sy_vmag, ~540 nm)", 'value': "sy_vmag"},
        {'label': "System Kepler Magnitude (sy_kepmag, ~600 nm)", 'value': "sy_kepmag"},
        {'label': "System r (Sloan) Magnitude (sy_rmag, ~622 nm)", 'value': "sy_rmag"},
        {'label': "System Gaia Magnitude (sy_gaiamag, ~673 nm)", 'value': "sy_gaiamag"},
        {'label': "System i (Sloan) Magnitude (sy_imag, ~763 nm)", 'value': "sy_imag"},
        {'label': "System I (Cousins) Magnitude (sy_icmag, ~786.5 nm)", 'value': "sy_icmag"},
        {'label': "System TESS Magnitude (sy_tmag, ~800 nm)", 'value': "sy_tmag"},
        {'label': "System z (Sloan) Magnitude (sy_zmag, ~905 nm)", 'value': "sy_zmag"},
        {'label': "System J (2MASS) Magnitude (sy_jmag, ~1.25 μm)", 'value': "sy_jmag"},
        {'label': "System H (2MASS) Magnitude (sy_hmag, ~1.65 μm)", 'value': "sy_hmag"},
        {'label': "System Ks (2MASS) Magnitude (sy_kmag, ~2,15 μm)", 'value': "sy_kmag"},
        {'label': "System W1 (WISE) Magnitude (sy_w1mag, ~3.4 μm)", 'value': "sy_w1mag"},
        {'label': "System W2 (WISE) Magnitude (sy_w2mag, ~4.6 μm)", 'value': "sy_w2mag"},
        {'label': "System W3 (WISE) Magnitude (sy_w3mag, ~12 μm)", 'value': "sy_w3mag"},
        {'label': "System W4 (WISE) Magnitude (sy_w4mag, ~22 μm)", 'value': "sy_w4mag"},
    ]
}
#creates dropdown options from the planet_features dictionary
dropdown_options = []
for category, options in planet_features.items():
    dropdown_options.append({
        'label': f'--- {category} ---',
        'value': f'header-{category}',
        'disabled': True
    })
    dropdown_options.extend(options)

app.layout = html.Div([
    html.H1("2d/3d Scatterplot"),

    html.H2("Data filters:"),
    dcc.Checklist(
        id='filter-checklist',
        options=[
            {'label': 'Has atmospheric data (Compare Planets only)', 'value': 'atmoData'},
            {'label': 'Default parameter set', 'value': 'default'},
            {'label': 'No controversial flag', 'value': 'noControv'},
            {'label': 'Water to Metal Density (Compare Planets only)', 'value': 'densRange'},
            {'label': 'In Target Star Catalog', 'value': 'target'},
            {'label': 'Include Solar System', 'value': 'solar'},
        ],
        value=['default', 'noControv'],  # Default values to filter by
        className="checkbox-container"
    ),
    html.Div([
        dcc.Checklist(
            id='host-star-toggle',
            options=[{'label': 'Filter by host star: ', 'value': 'enable'}],
            value=[],  # Start unchecked
            className="checkbox-container"
        ),
        dcc.Input(
            id='host-star-input',
            type='text',
            placeholder='Enter host star name',
            debounce=True,
            className="textbox-style"
        ),
    ], className="checkbox-container", style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'gap': '10px'}),

    html.Details([
        html.Summary("Planet type (Compare Planets only)"),
        dcc.Checklist(
            id='pltype-checklist',
            options=[
                {'label': 'Terrestrial', 'value': 'terrestrial'},
                {'label': 'Super-Earth', 'value': 'super_earth'},
                {'label': 'Neptune-like', 'value': 'neptune_like'},
                {'label': 'Gas Giant', 'value': 'gas_giant'},
                {'label': 'Unknown', 'value': 'unknown'},
                {'label': 'TBA', 'value': 'tba'},
            ],
            value=['terrestrial', 'super_earth', 'neptune_like', 'gas_giant', 'unknown', 'tba'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]),

    html.Details([
        html.Summary("Discovery method (Compare Planets only)"),
        dcc.Checklist(
            id='discmethod-checklist',
            options=[
                {'label': 'Transit', 'value': 'Transit'},
                {'label': 'Transit Timing Variations', 'value': 'Transit Timing Variations'},
                {'label': 'Eclipse Timing Variations', 'value': 'Eclipse Timing Variations'},
                {'label': 'Orbital Brightness Modulation', 'value': 'Orbital Brightness Modulation'},
                {'label': 'Radial Velocity', 'value': 'Radial Velocity'},
                {'label': 'Astrometry', 'value': 'Astrometry'},
                {'label': 'Imaging', 'value': 'Imaging'},
                {'label': 'Disc Kinematics', 'value': 'Disc Kinematics'},
                {'label': 'Microlensing', 'value': 'Microlensing'},
                {'label': 'Pulsar Timing', 'value': 'Pulsar Timing'},
                {'label': 'Pulsation Timing Variations', 'value': 'Pulsation Timing Variations'},
                {'label': 'Known Since Antiquity', 'value': 'Known Since Antiquity'},
                {'label': 'Unknown', 'value': 'null'},
            ],
            value=['Transit', 'Radial Velocity', 'Imaging', 'Eclipse Timing Variations', 'Microlensing', 'Pulsar Timing', 'Pulsation Timing Variations', 'Orbital Brightness Modulation', 'Transit Timing Variations', 'Astrometry', 'Disc Kinematics', 'Known Since Antiquity', 'null'],  # automatically selects all
            className="checkbox-container"
        ),
    ]),  

    html.Details([
        html.Summary("Discovery locale (Compare Planets only)"),
        dcc.Checklist(
            id='disclocale-checklist',
            options=[
                {'label': 'Space', 'value': 'Space'},
                {'label': 'Ground', 'value': 'Ground'},
                {'label': 'Multiple Locales', 'value': 'Multiple Locales'},
                {'label': 'Unknown', 'value': 'null'},
            ],
            value=['Space', 'Ground', 'Multiple Locales', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]), 
    
    html.Details([
        html.Summary("Harvard spectral classes"),
        dcc.Checklist(
            id='teff-checklist',
            options=[
                {'label': 'O type star (>33000 K)', 'value': 'O'},
                {'label': 'B type star (10000-33000 K)', 'value': 'B'},
                {'label': 'A type star (7300-10000 K)', 'value': 'A'},
                {'label': 'F type star (6000-7300 K)', 'value': 'F'},
                {'label': 'G type star (5300-6000 K)', 'value': 'G'},
                {'label': 'K type star (3900-5300 K, recommended)', 'value': 'K'},
                {'label': 'M type star (2300-3900 K)', 'value': 'M'},
                {'label': 'L type star (1300-2500 K)', 'value': 'L'},
                {'label': 'T type star (700-1300 K)', 'value': 'T'},
                {'label': 'Y type star (<700 K)', 'value': 'Y'},
                {'label': 'Wolf-Rayet star (>30000 K)', 'value': 'W'},
                {'label': 'White Dwarf (~5000-100000+ K)', 'value': 'D'},
                {'label': 'Unknown', 'value': 'null'},
            ],
            value=['O', 'B', 'A', 'F', 'G', 'K', 'M', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]),

    html.Details([
        html.Summary("Yerkes spectral classes"),
        dcc.Checklist(
            id='lum-checklist',
            options=[
                # {'label': 'Hypergiant (0)', 'value': '0'},
                # {'label': 'Supergiant (I)', 'value': 'I'},
                {'label': 'Bright Giant (II)', 'value': 'II'},
                {'label': 'Giant (III)', 'value': 'III'},
                {'label': 'Subgiant (IV)', 'value': 'IV'},
                {'label': 'Main-sequence/Dwarf (V, recommended)', 'value': 'V'},
                {'label': 'Subdwarf (VI)', 'value': 'VI'},
                # {'label': 'White Dwarf (VII)', 'value': 'VII'},
                {'label': 'Unknown', 'value': 'null'}
            ],
            value=['0', 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]),
    
    html.Details([
        html.Summary("Stellar metallicity ratio"),
        dcc.Checklist(
            id='met-checklist',
            options=[
                {'label': 'Iron abundance (recommended)', 'value': '[Fe/H]'},
                {'label': 'General metal content', 'value': '[M/H]'},
                {'label': 'Unknown', 'value': 'null'},
            ],
            value=['[Fe/H]', '[M/H]', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]), 

    html.H2("Select features:"),
    dcc.Dropdown(
        id='feature-dropdown-one',
        options=dropdown_options,
        value="pl_rade",  # Default value
        className="dropdown-container",
        placeholder="First feature"
    ),
    dcc.RadioItems(
        id="scale-radioitems-one",
        options=[
            {'label': 'Linear', 'value': 'linear'},
            {'label': 'Log', 'value': 'log'}
        ],
        value='linear',
        className="checkbox-container"
    ),
    dcc.Dropdown(
        id='feature-dropdown-two',
        options=dropdown_options,
        value="pl_bmasse",  # Default value
        className="dropdown-container",
        placeholder="Second feature"
    ),
    dcc.RadioItems(
        id="scale-radioitems-two",
        options=[
            {'label': 'Linear', 'value': 'linear'},
            {'label': 'Log', 'value': 'log'}
        ],
        value='linear',
        className="checkbox-container"
    ),
    dcc.Dropdown(
        id='feature-dropdown-three',
        options=dropdown_options,
        value="pl_dens",  # Default value
        className="dropdown-container",
        placeholder="Third feature"
    ),
    dcc.RadioItems(
        id="scale-radioitems-three",
        options=[
            {'label': 'Linear', 'value': 'linear'},
            {'label': 'Log', 'value': 'log'}
        ],
        value='linear',
        className="checkbox-container"
    ),

    html.H2("Group by:"),
    dcc.RadioItems(
        id="colorcode-radioitems",
        options=[
            {'label': 'Planet Type (Compare Planets only)', 'value': 'pl_type'},
            {'label': 'Harvard Spec. Class', 'value': 'st_teffclass'},
            {'label': 'Yerkes Spec. Class', 'value': 'st_lumclass'},
            {'label': 'Stellar Metallicity', 'value': 'st_metratio'},
            {'label': 'Discovery Method (Compare Planets only)', 'value': 'discoverymethod'},
            {'label': 'Discovery Locale (Compare Planets only)', 'value': 'disc_locale'},
            # {'label': 'Host Star', 'value': 'hostname'}, # too much computational power.
        ],  
        value="pl_type",
        className="checkbox-container"
    ),

    html.H2("Extra options:"),
    dcc.Checklist(
        id='extra-checklist',
        options=[
            {'label': html.Span(['Simplify planet types and discovery methods (Compare Planets only)'], style={'fontWeight': 'bold'}), 'value': 'simple'},
            {'label': html.Span(['Estimate missing values'], style={'fontWeight': 'bold'}), 'value': 'estimate'},
            {'label': html.Span(['Compare stars'], style={'fontWeight': 'bold'}), 'value': 'suns'},
        ],
        value=[],  # Start unchecked
        className="checkbox-container"
    ),

    html.Button('Update Graph', id='update-button', n_clicks=1),

    dcc.Tabs(id="tabs", value="tab-1", children=[
        dcc.Tab(label="2d Scatterplot", value="tab-1", className="tab"),
        dcc.Tab(label="3d Scatterplot", value="tab-2", className="tab"),
    ]),
    html.Div(id="tabs-content", children=[
        dcc.Graph(id="2d-plot", figure={}, style={"display": "block"}),
        dcc.Graph(id="3d-plot", figure={}, style={"display": "none"}),
    ]),  # This will hold the selected graph

    html.P('NaN info:', id='naninfo-box'),

    html.P("Data sourced from:"),
    html.Div([
        html.A("https://www.doi.org/10.26133/NEA12", href="https://www.doi.org/10.26133/NEA12", target="_blank"),
        html.Br(),
        html.A("https://www.doi.org/10.26133/NEA36", href="https://www.doi.org/10.26133/NEA36", target="_blank"),
        html.Br(),
        html.A("https://science.nasa.gov/exoplanets/exoplanet-catalog/", href="https://science.nasa.gov/exoplanets/exoplanet-catalog/", target="_blank"),
        html.Br(),
        html.A("https://science.nasa.gov/exoplanets/target-star-catalog/", href="https://science.nasa.gov/exoplanets/target-star-catalog/", target="_blank"),
    ]),
    
    html.Button("Reset Settings", id="reset-button", n_clicks=0),

    dcc.Store(id='settings-store', storage_type='local'),
    dcc.Store(id='rehydration-complete', data=False),
    dcc.Store(id='init-flag', data=False)
], style={'color': 'black', 'font-family': 'Arial', 'backgroundColor': 'white', 'padding': '20px', 'text-align': 'center'})

def update_graph(update, filters, extra, featureone, featuretwo, featurethree, scaleone, scaletwo, scalethree, startoggle, starinput, colorcode, pltype, discmethod, disclocale, teff, lum, met):
    # error handling
    if any(f is None for f in [featureone, featuretwo, featurethree]):
        raise dash.exceptions.PreventUpdate
    planet_feature_values = {option['value'] for option in planet_features['Planet (Compare Planets only)']}
    if 'suns' in extra and (any(f in planet_feature_values for f in [featureone, featuretwo, featurethree])):
        return go.Figure(), go.Figure(), "ERROR: Cannot use planet features when using star data."
    if 'suns' in extra and (colorcode == "pl_type" or colorcode == 'disc_locale' or colorcode == 'discoverymethod'):
        return go.Figure(), go.Figure(), "ERROR: Cannot filter by planet type when using star data."
    if featureone==featuretwo or featureone==featurethree or featuretwo==featurethree:
        return go.Figure(), go.Figure(), "ERROR: Features must be different."
    
    # Load dataset
    if 'estimate' in extra:
        df = pl_es.copy()
    else:
        df = pl_s.copy()

    # Adds solar system data if requested
    if 'solar' in filters:
        if 'suns' in extra:
            first_solar_entry = solar_planets.iloc[[0]].copy()  # Get just the first row as a DataFrame
            df = pd.concat([pl_s.copy(), first_solar_entry], ignore_index=True)
        else:
            df = pd.concat([pl_s.copy(), solar_planets.copy()], ignore_index=True) # type: ignore
    
    # turns dates into seconds since epoch
    def normalize_timestamp_column(df, feature, epoch=pd.Timestamp("1970-01-01")):
        if feature in {"rowupdate", "releasedate"} and feature in df.columns:
            df.loc[:, feature] = (
                pd.to_datetime(df[feature], errors="coerce")
                .subtract(epoch)
                .dt.total_seconds()
                .astype("Int64")  # Nullable integer type
            )
    normalize_timestamp_column(df, featureone)
    normalize_timestamp_column(df, featuretwo)
    normalize_timestamp_column(df, featurethree)

    # Cuts out features below minimum or above maximum
    if not 'suns' in extra:
        if 'atmoData' in filters:
            df = df[df['pl_name'].isin(at_s['pl_name']) | df["is_solar"] == True]
        if 'densRange' in filters:
            df=df[(df['pl_dens'] > 1) & (df['pl_dens'] < 5.6)] # Only gets planets with a density between water and metallic iron
        df = df[df['pl_type'].isin(pltype)]  
        if 'simple' in extra:
            df["pl_type"] = df["pl_type"].replace({"super_earth": "terrestrial", "neptune_like": "gas_giant"}) 
            df["discoverymethod"] = df["discoverymethod"].replace({"Transit Timing Variations": "Transit", "Eclipse Timing Variations": "Transit", "Orbital Brightness Modulation": "Transit", "Astrometry": "Radial Velocity", "Pulsation Timing Variations": "Pulsar Timing", "Disc Kinematics": "Imaging"}) 
        if 'null' in discmethod:
            df = df[df['discoverymethod'].isin(discmethod) | df['discoverymethod'].isna()]
        else:
            df = df[df['discoverymethod'].isin(discmethod)]
        if 'null' in disclocale:
            df = df[df['disc_locale'].isin(disclocale) | df['disc_locale'].isna()]
        else:
            df = df[df['disc_locale'].isin(disclocale)]
    
    if 'null' in teff:
        df = df[df['st_teffclass'].isin(teff) | df['st_teffclass'].isna()] 
    else:
        df = df[df['st_teffclass'].isin(teff)]   
    if 'null' in lum:
        df = df[df['st_lumclass'].isin(lum) | df['st_lumclass'].isna()] 
    else:
        df = df[df['st_lumclass'].isin(lum)]
    if 'default' in filters:
        df = df[df['default_flag'] == True]
    if 'noControv' in filters:
        df = df[df['pl_controv_flag'] == False]
    if 'target' in filters:
        df = df[df['hostname'].isin(target_stars) | df["is_solar"] == True] # type: ignore
    if 'null' in met:
        df = df[df['st_metratio'].isin(met) | df['st_metratio'].isna()]
    else:
        df = df[df['st_metratio'].isin(met)]
    if startoggle and starinput.strip():
        # Normalize both sides for safe matching
        df = df[df['hostname'].str.lower() == starinput.strip().lower()]

    # naninfo stuff
    naninfo = f"Total Entries: {len(df)}\n"
    threshold = len(df) / 2
    naninfo_list = []
    for feature in [featureone, featuretwo, featurethree]:
        count = df[feature].isna().sum()
        flag = " (WARNING: >50% missing)" if count > threshold else ""
        naninfo_list.append(
            f"{feature}{flag}: {count} NaNs, mean {df[feature].mean():.2f}, median {df[feature].median():.2f}, variance {df[feature].var():.2f}"
        )

    # Compose one-liner NaN info
    naninfo = naninfo + "\n".join(naninfo_list)

    # Select features to be evaluated (keeps certain others for hover info)
    if 'suns' in extra:
        df = df[[featureone] + [featuretwo] + [featurethree] +
            ["hostname"] + ["is_solar"] + ["st_spectype"] + ["st_teffclass"] + ["st_lumclass"] + ["st_metratio"]
        ]
        # Group by hostname and get the mode for each feature
        def get_mode(series):
            mode_vals = series.mode()
            if not mode_vals.empty:
                return mode_vals.iloc[0]  # Return the first mode (if multiple)
            return np.nan  # Or return None, depending on preference
        df = df.groupby("hostname", as_index=False).agg(get_mode)
    else:
        df = df[["pl_name"] + ["pl_type"] +
            ["discoverymethod"] + ["disc_refname"] + ["disc_locale"] + ["disc_facility"] + ["disc_telescope"] + ["disc_instrument"] +
            ["hostname"] + ["is_solar"] + ["st_spectype"] + ["st_teffclass"] + ["st_lumclass"] + ["st_metratio"] +
            [featureone] + [featuretwo] + [featurethree]
        ]

    # Returns empty graphs if filtered dataset is empty
    if df.empty:
        return go.Figure(), go.Figure(), "ERROR: filtered dataset is empty"
    
    #colorcode logic
    df[colorcode] = df[colorcode].fillna("unknown") # Handle NaNs in colorcode by assigning 'unknown'
    color_map = {
        "terrestrial": "blue", "super_earth": "red", "neptune_like": "green", "gas_giant": "purple", "tba": "black",
        "II": "yellow", "III": "orange", "IV": "green", "V": "blue", "VI": "black",
        "B": "darkblue", "A": "royalblue", "F": "seagreen", "G": "yellow", "K": "orange", "M": "red",
        "Transit": "royalblue", "Transit Timing Variations": "slateblue", "Eclipse Timing Variations": "mediumslateblue", "Orbital Brightness Modulation": "cornflowerblue", "Radial Velocity": "crimson", "Astrometry": "firebrick", "Imaging": "seagreen", "Disc Kinematics": "mediumseagreen", "Microlensing": "darkgoldenrod", "Pulsar Timing": "mediumvioletred", "Pulsation Timing Variations": "orchid", "Known Since Antiquity": "black",
        "Space": "deepskyblue", "Ground": "sienna", "Multiple Locales": "mediumorchid",
        "[Fe/H]": "steelblue", "[M/H]": "darkcyan",
        "unknown": "gray",
    }

    # relabels discrete columns for color coding
    if colorcode in df.columns and df[colorcode].dtype == "object":
        # Count values
        label_counts = df[colorcode].value_counts().to_dict()

        # Create a mapping: e.g., "terrestrial" → "terrestrial (42)"
        labeled_with_counts = {
            key: f"{key} ({label_counts.get(key, 0)})" for key in df[colorcode].unique()
        }

        # Apply the relabeling
        df["colorcode_labeled"] = df[colorcode].map(labeled_with_counts)
        colorcode_use = "colorcode_labeled"

        # Generate fallback color iterator from Plotly palette
        default_colors = px.colors.qualitative.Plotly
        default_color_cycle = iter(default_colors)

        color_map_labeled = {}
        for key in df[colorcode].unique():
            label = labeled_with_counts[key]
            if key in color_map:
                color_map_labeled[label] = color_map[key]
            else:
                color_map_labeled[label] = next(default_color_cycle)
    else:
        # For continuous or numeric colorcodes, no relabeling
        colorcode_use = colorcode
        color_map_labeled = color_map
    
    # Increases size of solar system planets
    df["is_solar"] = df["is_solar"].map(lambda x: bool(x) if pd.notnull(x) else False)
    df["marker_size"] = df["is_solar"].fillna(False).apply(lambda x: 3 if x else 1)

    # hovertext info
    if 'suns' in extra:
        hover_cols = ['hostname']
    else:
        hover_cols = ['pl_name', 'hostname', 'discoverymethod', 'disc_refname', 'disc_locale', 'disc_facility', 'disc_telescope', 'disc_instrument']
    
    # gets labels for features
    feature_labelone = next(
        (opt['label'] for opt in dropdown_options if not opt.get('disabled') and opt['value'] == featureone),
        feature  # fallback if not found
    )
    feature_labeltwo = next(
        (opt['label'] for opt in dropdown_options if not opt.get('disabled') and opt['value'] == featuretwo),
        feature  # fallback if not found
    )
    feature_labelthree = next(
        (opt['label'] for opt in dropdown_options if not opt.get('disabled') and opt['value'] == featurethree),
        feature  # fallback if not found
    )

    # Generate Plotly graphs
    plot_2d = px.scatter(df, x=featureone, y=featuretwo, color=colorcode_use, color_discrete_map=color_map_labeled if df[colorcode_use].dtype == "object" else None, size="marker_size", hover_data=hover_cols)
    plot_2d.update_traces(
        hoverlabel=dict(
            font=dict(color="black"),
            bgcolor="white",
        ),
    )
    plot_2d.update_layout(xaxis_type=scaleone, yaxis_type=scaletwo)
    plot_2d.update_layout(
        legend=dict(
            font=dict(size=20),  # Adjust size here
            itemsizing='constant'  # Keeps marker size consistent
        )
    )
    plot_2d.update_xaxes(title_text=feature_labelone)
    plot_2d.update_yaxes(title_text=feature_labeltwo)
    
    plot_3d = px.scatter_3d(df, x=featureone, y=featuretwo, z=featurethree, color=colorcode_use, color_discrete_map=color_map_labeled if df[colorcode_use].dtype == "object" else None, size="marker_size", hover_data=hover_cols)
    plot_3d.update_traces(
        hoverlabel=dict(
            font=dict(color="black"),
            bgcolor="white",
        ),
    )
    plot_3d.update_layout(scene=dict(
        xaxis=dict(type=scaleone),
        yaxis=dict(type=scaletwo),
        zaxis=dict(type=scalethree)
    ))
    plot_3d.update_layout(
        legend=dict(
            font=dict(size=20),  # Adjust size here
            itemsizing='constant'  # Keeps marker size consistent
        )
    )
    plot_3d.update_layout(
        scene=dict(
            xaxis=dict(title=feature_labelone),
            yaxis=dict(title=feature_labeltwo),
            zaxis=dict(title=feature_labelthree)
        )
    )

    return plot_2d, plot_3d, html.Pre(naninfo)

# changes visible graph based on tab selection
@app.callback(
    [Output('2d-plot', 'style'),
     Output('3d-plot', 'style'),],
    [Input("tabs", "value")]
)
def update_tabs(tab):
    styles = {
        "tab-1": [{"display": "block"}, {"display": "none"}],
        "tab-2": [{"display": "none"}, {"display": "block"}],
    }
    return styles.get(tab, [{"display": "block"}, {"display": "none"}])
  
# updates graphs based on user input or stored settings
@app.callback(
    [Output('2d-plot', 'figure'),
     Output('3d-plot', 'figure'),
     Output('naninfo-box', 'children'),
     Output('settings-store', 'data')],
    [Input('update-button', 'n_clicks'),
     Input('settings-store', 'data')],
    [State('filter-checklist', 'value'),
     State('extra-checklist', 'value'),
     State('feature-dropdown-one', 'value'),
     State('feature-dropdown-two', 'value'),
     State('feature-dropdown-three', 'value'),
     State('scale-radioitems-one', 'value'),
     State('scale-radioitems-two', 'value'),
     State('scale-radioitems-three', 'value'),
     State('host-star-toggle', 'value'),
     State('host-star-input', 'value'),
     State('colorcode-radioitems', 'value'),
     State('pltype-checklist', 'value'),
     State('discmethod-checklist', 'value'),
     State('disclocale-checklist', 'value'),
     State('teff-checklist', 'value'),
     State('lum-checklist', 'value'),
     State('met-checklist', 'value'),]
)
def update_figures(n_clicks, settings, *states):
    # Check if the callback was triggered by a button click or settings change
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate
    trigger = ctx.triggered[0]['prop_id']

    if trigger == 'update-button.n_clicks':
        # Use current inputs
        plot_2d, plot_3d, naninfo = update_graph(n_clicks, *states)
        settings_dict = dict(zip([
            'filter', 'extra', 'featureone', 'featuretwo', 'featurethree',
            'scaleone', 'scaletwo', 'scalethree', 'hosttoggle', 'hostinput',
            'colorcode', 'pltype', 'discmethod', 'disclocale', 'teff', 'lum', 'met'
        ], states))
        return plot_2d, plot_3d, naninfo, settings_dict

    elif trigger == 'settings-store.data':
        # Use stored settings, not states
        if not settings:
            raise dash.exceptions.PreventUpdate
        plot_2d, plot_3d, naninfo = update_graph(0, *unpack_settings(settings))
        return plot_2d, plot_3d, naninfo, dash.no_update

# Restore settings from local storage when the app starts
@app.callback(
    [Output('filter-checklist', 'value'),
     Output('extra-checklist', 'value'),
     Output('feature-dropdown-one', 'value'),
     Output('feature-dropdown-two', 'value'),
     Output('feature-dropdown-three', 'value'),
     Output('scale-radioitems-one', 'value'),
     Output('scale-radioitems-two', 'value'),
     Output('scale-radioitems-three', 'value'),
     Output('host-star-toggle', 'value'),
     Output('host-star-input', 'value'),
     Output('colorcode-radioitems', 'value'),
     Output('pltype-checklist', 'value'),
     Output('discmethod-checklist', 'value'),
     Output('disclocale-checklist', 'value'),
     Output('teff-checklist', 'value'),
     Output('lum-checklist', 'value'),
     Output('met-checklist', 'value'),
     Output('init-flag', 'data')],
    Input('settings-store', 'data'),
    State('init-flag', 'data'),
    prevent_initial_call="initial_duplicate"
)
def restore_settings(settings, already_initialized):
    if not settings or already_initialized:
        raise dash.exceptions.PreventUpdate
    return (
        settings['filter'], settings['extra'], settings['featureone'],
        settings['featuretwo'], settings['featurethree'],
        settings['scaleone'], settings['scaletwo'], settings['scalethree'],
        settings['hosttoggle'], settings['hostinput'],
        settings['colorcode'], settings['pltype'],
        settings['discmethod'], settings['disclocale'], settings['teff'],
        settings['lum'], settings['met'], True
    )

# Unpack settings from the stored dictionary
def unpack_settings(settings):
    return (
        settings['filter'], settings['extra'], settings['featureone'],
        settings['featuretwo'], settings['featurethree'],
        settings['scaleone'], settings['scaletwo'], settings['scalethree'],
        settings['hosttoggle'], settings['hostinput'],
        settings['colorcode'], settings['pltype'],
        settings['discmethod'], settings['disclocale'], settings['teff'],
        settings['lum'], settings['met'],
    )

# Clear the store when the reset button is clicked
@app.callback(
    Output('settings-store', 'data', allow_duplicate=True),
    Input('reset-button', 'n_clicks'),
    prevent_initial_call=True
)
def clear_store(n):
    return None  # or {}

if __name__ == '__main__':
    app.run(debug=True, port=8054)
    print("running on localhost:8054")

    # Note: add an error handler for any of the features matching.

running on localhost:8054
